In [2]:
import pandas as pd

In [8]:
df = pd.read_csv('../data/misc/daily_portfolio_value.csv')
df.head()

,date,portfolio_value,ADA-USD,ETH-USD,LINK-USD,LTC-USD,SOL-USD,UNI-USD,XRP-USD,ADI,...,LRCX,MCHP,MCO,MOH,ODFL,PCG,PHM,PLD,STZ,TSLA
0,2025-03-01,1.000000e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2025-03-02,1.000000e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2025-03-03,9.999924e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2025-03-04,9.992645e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2025-03-05,1.000049e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
trades = pd.read_csv('../data/misc/trade_log.csv')
trades.head()

,asset,date,action,price,units,signed_value
0,LII,2025-03-03,BUY_NEW,586.979980,5.195480e+01,-30496.424882
1,LII,2025-03-06,BUY_ADD,583.900024,2.774973e-01,-162.030664
2,UNI-USD,2025-03-10,BUY_NEW,0.000158,5.797783e+06,-916.049651
3,ADI,2025-03-10,BUY_NEW,216.259995,1.330952e+02,-28783.165230
4,ADI,2025-03-12,BUY_ADD,206.619995,5.814890e+00,-1201.472533


In [10]:
## Annual Return

df.iloc[-1]['portfolio_value'] / df.iloc[0]['portfolio_value'] - 1


-0.0047309456202055955

In [16]:
## Sharpe Ratio
pct_change = df['portfolio_value'].pct_change()[1:]
(pct_change.mean() - 0.0431) / pct_change.std()

-1.0227256352844156

In [22]:
# Max Drawdown
peak = df['portfolio_value'].cummax()
drawdown = (df['portfolio_value'] - peak) / peak
f'{drawdown.min() * 100:.2f}%'

'-10.56%'

In [97]:
# Win Rate

d = {}
for row in trades.iterrows():
    entry = row[1]

    if entry['action'] == 'BUY_NEW':
        d[entry['asset']] = (entry['date'], entry['price'])
        
    elif entry['action'] == 'BUY_ADD':
        assert entry['asset'] in d
        d[entry['asset']] = (entry['date'], d[entry['asset']][1] + entry['price'])
        
    elif entry['action'] in ['SELL_SL', 'SELL_TP']:
        assert entry['asset'] in d
        d[entry['asset']] = (entry['date'], d[entry['asset']][1] - entry['price'])
        
d

{'LII': ('2025-03-28', 1757.02001953125),
 'UNI-USD': ('2025-03-29', 0.0001659999979892),
 'ADI': ('2025-03-28', 201.55999755859372),
 'MCHP': ('2025-03-28', 99.70000076293945),
 'MOH': ('2025-03-21', 309.92999267578125),
 'XRP-USD': ('2025-03-28', 0.14495086669921875)}

In [24]:
# Annual Volatility
f'{pct_change.std() * (252 ** 0.5) * 100:.2f}%'

'65.86%'

In [31]:
# Drawdown Patterns
is_drawdown = drawdown < 0
(is_drawdown & (~is_drawdown.shift(1).fillna(False))).sum()

C:\Users\JW\AppData\Local\Temp\ipykernel_20372\226863221.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  (is_drawdown & (~is_drawdown.shift(1).fillna(False))).sum()


3

In [33]:
# Number of Trades
len(trades)

18

In [60]:
# Average Holding Period

d = {}
for row in trades.iterrows():
    entry = row[1]

    if entry['action'] == 'BUY_NEW':
        d[entry['asset']] = (entry['date'], entry['price'])
        
    elif entry['action'] == 'BUY_ADD':
        assert entry['asset'] in d
        d[entry['asset']] = (entry['date'], d[entry['asset']][1] + entry['price'])
        
    elif entry['action'] in ['SELL_SL', 'SELL_TP']:
        assert entry['asset'] in d
        d[entry['asset']] = (entry['date'], d[entry['asset']][1] - entry['price'])

In [95]:
# Profit per Trade

'''
I will consider the price at the end of the trading period 
of the assets not sold as part of the profit.
'''

leftover_value = df.iloc[-1][d.keys()]
leftover_value = leftover_value[leftover_value > 0]

pd.concat([trades['signed_value'], leftover_value]).mean()

-203.82718204234732